In [1]:
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from os import walk
import sys
import os.path
import glob
import os

In [2]:
# folder 안에 파일 이름 저장
mp3_list =[]
folder = [j for i,j,k in walk(r'../fma_medium')][0]
temp = [mp3_list.extend(k) for i,j,k in walk(r'../fma_medium')]
music_list = [i[:10] for i in mp3_list[2:]]

In [73]:
genreslist = pd.read_csv('./fma_metadata/genres.csv')
genreslist['title'] = [i.lower() for i in genreslist['title']]
print('#Genres = ',len(genreslist))

#Genres =  163


In [77]:
genreslist[genreslist['title']=='rap']

,genre_id,#tracks,parent,title,top_level
142,539,638,21,rap,21


In [4]:
genredict = {k:v for v,k in zip(genreslist['title'],genreslist['genre_id'])}

In [84]:
dataset = pd.read_pickle('FMA_M_multilabeldataset')

In [3]:
genreslist = pd.read_csv('./fma_metadata/genres.csv')
genreslist['title'] = [i.lower() for i in genreslist['title']]
print('#Genres = ',len(genreslist))

#Genres =  163


In [4]:
# Glove vocab 안에 있는 genre dict
wordvec = {}
f=open("glove.6B.100d.txt",encoding='utf-8')
for i in f:
    word = i.split()[0]
    if word in list(genreslist['title']):
        wordvec[word] = i.split()[1:]
print('#Genres = ',len(wordvec.keys()))
print('Glove vocab에 포함되지 않는 장르')
for i,j in enumerate(genreslist['title']):
    if j not in list(wordvec.keys()):
        print(i,j)
        genreslist = genreslist.drop([i])

#Genres =  97
Glove vocab에 포함되지 않는 장르
7 old-time / historic
12 easy listening
13 soul-rnb
15 sound effects
21 audio collage
25 field recordings
28 psych-folk
30 jazz: vocal
33 ambient electronic
34 radio art
35 loud-rock
36 latin america
38 free-folk
39 noise-rock
40 psych-rock
42 electro-punk
46 no wave
48 experimental pop
50 reggae - dub
55 new wave
59 freak-folk
60 jazz: out
62 alternative hip-hop
63 death-metal
64 middle east
69 space-rock
75 spoken weird
77 black-metal
79 easy listening: vocal
81 asia-far east
82 n. indian traditional
83 south indian traditional
88 big band/swing
89 british folk
93 minimal electronic
94 breakcore - hard
95 sound poetry
96 20th century classical
98 talk radio
99 north african
100 sound collage
104 musique concrete
106 new age
109 chip music
112 composed music
113 drum & bass
117 synth pop
119 deep funk
120 spoken word
122 bigbeat
124 radio theater
126 rock opera
128 chamber music
129 choral music
132 musical theater
134 skweee
135 western swing
139

In [86]:
genredict = {k:v for k,v in zip(genreslist['genre_id'],genreslist['title'])}
tmp = [(i,genredict[i]) for i in list(dataset['Genres']) ]
genrewordDF = pd.DataFrame(tmp)
genrewordDF.columns = ['Genre','Genreword']
dataset = pd.concat([dataset,genrewordDF['Genreword']],axis=1)

In [92]:
tmp = []
for i in list(dataset['Genreword']):
    if i in list(wordvec.keys()):
        tmp.append((i,wordvec[i]))
    else:
        tmp.append((i,np.nan))

In [94]:
tmpDF = pd.DataFrame(tmp)
tmpDF.columns = ['Genreword','Gen2vec']
dataset = pd.concat([dataset,tmpDF['Gen2vec']],axis=1)

In [97]:
import pickle
with open('FMA_M_multilabeldataset_final', 'wb') as f:
    pickle.dump(dataset, f)

In [8]:
dataset = pd.read_pickle('FMA_M_multilabeldataset_final')

In [10]:
wordvecDF = pd.DataFrame(wordvec)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
wordvec.keys()

dict_keys(['country', 'international', 'house', 'french', 'europe', 'radio', 'african', 'indian', 'interview', 'rock', 'spanish', 'pacific', 'industrial', 'turkish', 'latin', 'christmas', 'dance', 'holiday', 'metal', 'electronic', 'pop', 'brazilian', 'opera', 'comedy', 'jazz', 'spoken', 'classical', 'blues', 'poetry', 'progressive', 'folk', 'noise', 'experimental', 'celtic', 'symphony', 'soundtrack', 'instrumental', 'jungle', 'garage', 'compilation', 'gospel', 'rap', 'punk', 'balkan', 'surf', 'lounge', 'funk', 'disco', 'drone', 'bollywood', 'hip-hop', 'hardcore', 'novelty', 'singer-songwriter', 'salsa', 'tango', 'bluegrass', 'techno', 'ambient', 'glitch', 'thrash', 'americana', 'sludge', 'avant-garde', 'flamenco', 'polka', 'banter', 'improv', 'goth', 'rockabilly', 'minimalism', 'post-punk', 'klezmer', 'dubstep', 'cumbia', 'fado', 'lo-fi', 'post-rock', 'electroacoustic', 'grindcore', 'breakbeat', 'idm', 'wonky', 'afrobeat', 'krautrock', 'downtempo', 'indie-rock', 'trip-hop', 'shoegaze',

In [50]:
cosine_similarity([wordvec['hip-hop'],wordvec['rap']])[0][1]

0.8027884404632174

In [71]:
hcorrword = []
for w in wordvec.keys():
    for wp in wordvec.keys():
        if w==wp:
            pass
        elif cosine_similarity([wordvec[w],wordvec[wp]])[0][1] > 0.7:
            hcorrword.append((w,wp,cosine_similarity([wordvec[w],wordvec[wp]])[0][1]))

In [72]:
hcorrword

[('country', 'europe', 0.7167598346961536),
 ('french', 'spanish', 0.758211955268014),
 ('europe', 'country', 0.7167598346961536),
 ('rock', 'pop', 0.7647685556700637),
 ('rock', 'punk', 0.7609220634558033),
 ('spanish', 'french', 0.758211955268014),
 ('christmas', 'holiday', 0.8736868425460984),
 ('dance', 'folk', 0.7121563748332252),
 ('holiday', 'christmas', 0.8736868425460984),
 ('pop', 'rock', 0.7647685556700637),
 ('pop', 'rap', 0.7511845693609087),
 ('pop', 'punk', 0.715317649417896),
 ('pop', 'hip-hop', 0.7274856864373817),
 ('opera', 'symphony', 0.7103985305009026),
 ('jazz', 'blues', 0.7819991899477121),
 ('jazz', 'folk', 0.7261057802695668),
 ('blues', 'jazz', 0.7819991899477121),
 ('folk', 'dance', 0.7121563748332252),
 ('folk', 'jazz', 0.7261057802695668),
 ('symphony', 'opera', 0.7103985305009026),
 ('soundtrack', 'compilation', 0.8114907429053264),
 ('compilation', 'soundtrack', 0.8114907429053264),
 ('rap', 'pop', 0.7511845693609087),
 ('rap', 'hip-hop', 0.8027884404632

In [43]:
rap, indie-rock, post-punk, shoegaze, hardcore

numpy.ndarray